In [1]:
import pandas as pd
import numpy as np
import pathlib
import os
import sys
import glob
import tensorflow as tf
# from tensorflow.keras.applications.inception_v3 import preprocess_input
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from IPython.display import Image, display
import cv2
%matplotlib inline

/home/ccuulinay/anaconda3/envs/tf2_0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ccuulinay/anaconda3/envs/tf2_0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ccuulinay/anaconda3/envs/tf2_0/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ccuulinay/anaconda3/envs/tf2_0/lib

## Preprocess data

In [2]:
data_path = "./data"
data_p = pathlib.Path(data_path)

In [3]:
meta_table = pd.read_csv("./_filename_dict.csv")

In [4]:
# Filter is_readable_image == False
f_table = meta_table[(meta_table['is_readable_image']) & (meta_table['is_completed_image'])].copy()
# f_table = f_table[f_table['_format'] != ".gif"].copy()
f_table.reset_index(drop=True, inplace=True)
f_table.fillna("", inplace=True)

In [5]:
"""filenames = ['./data/' + fname for fname in f_table['_filename'].tolist()]

def preprocess_image(image):
    image = tf.image.decode_jpeg(image)
    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

def get_corrupted_image_files(file_list):
    cr_list = []
    
    for i, f in enumerate(filenames):
        try:
            load_and_preprocess_image(f)
        except Exception as e:
            cr_list.append((i, f))
            
    return cr_list

corrupted_images = get_corrupted_image_files(filenames)

corrupted_images_index = [i[0] for i in corrupted_images]

f_table = f_table.iloc[~f_table.index.isin(corrupted_images_index)].copy()

f_table.reset_index(drop=True, inplace=True)
f_table.fillna("", inplace=True)"""

'filenames = [\'./data/\' + fname for fname in f_table[\'_filename\'].tolist()]\n\ndef preprocess_image(image):\n    image = tf.image.decode_jpeg(image)\n    return image\n\ndef load_and_preprocess_image(path):\n    image = tf.io.read_file(path)\n    return preprocess_image(image)\n\ndef get_corrupted_image_files(file_list):\n    cr_list = []\n    \n    for i, f in enumerate(filenames):\n        try:\n            load_and_preprocess_image(f)\n        except Exception as e:\n            cr_list.append((i, f))\n            \n    return cr_list\n\ncorrupted_images = get_corrupted_image_files(filenames)\n\ncorrupted_images_index = [i[0] for i in corrupted_images]\n\nf_table = f_table.iloc[~f_table.index.isin(corrupted_images_index)].copy()\n\nf_table.reset_index(drop=True, inplace=True)\nf_table.fillna("", inplace=True)'

In [41]:
f_table['sample'].value_counts()


皮鞋      100
电池      100
糖果       99
书本       99
油桶       99
       ... 
编织袋      54
硬果壳      53
玉米衣      53
尼龙制品     47
家养植物      7
Name: sample, Length: 154, dtype: int64

In [6]:
f_table['level0'].value_counts()

湿垃圾     3703
可回收物    3629
有害垃圾    2806
干垃圾     2407
Name: level0, dtype: int64

In [42]:
f_table.head()

,_filename,_format,sample,spider_id,level0,level1,is_readable_image,is_completed_image
0,干垃圾_编织袋_36.jpeg,.jpeg,编织袋,36,干垃圾,,True,True
1,湿垃圾_食材废料_水产_91.jpg,.jpg,水产,91,湿垃圾,食材废料,True,True
2,湿垃圾_过期食品_肉干_7.jpg,.jpg,肉干,7,湿垃圾,过期食品,True,True
3,干垃圾_橡皮泥_20.jpg,.jpg,橡皮泥,20,干垃圾,,True,True
4,有害垃圾_废含汞温度计、废含汞血压计_水银体温计_84.jpg,.jpg,水银体温计,84,有害垃圾,废含汞温度计、废含汞血压计,True,True


In [68]:
filenames = ['./data/' + fname for fname in f_table['_filename'].tolist()]
lv0_labels = f_table['level0'].tolist()

In [69]:
labelencoder = preprocessing.LabelEncoder()
lv0_labels_encoded = labelencoder.fit_transform(lv0_labels)

In [70]:
# print(labelencoder.transform(['可回收物', "有害垃圾", "湿垃圾", "干垃圾"]))
print(len(labelencoder.classes_))
lv0_n_classes = len(labelencoder.classes_)

4


In [71]:
raw_filenames, test_filenames, raw_labels, test_labels = train_test_split(filenames
                                                                        , lv0_labels_encoded
                                                                        , train_size=0.9
                                                                        , random_state=42)

train_filenames, val_filenames, train_labels, val_labels = train_test_split(raw_filenames
                                                                        , raw_labels
                                                                        , train_size=0.9
                                                                        , random_state=42)

In [72]:
print(len(train_filenames), len(train_labels))
print(len(val_filenames), len(val_labels))
print(len(test_filenames), len(test_labels))

10161 10161
1129 1129
1255 1255


In [73]:
train_data = tf.data.Dataset.from_tensor_slices(
  (tf.constant(train_filenames), tf.constant(train_labels))
)
val_data = tf.data.Dataset.from_tensor_slices(
  (tf.constant(val_filenames), tf.constant(val_labels))
)

In [74]:
IMAGE_SIZE = 149
BATCH_SIZE = 32

In [75]:
def _parse_fn(filename, label):
    try:
        # read file as byte
        image_string = tf.io.read_file(filename)
        # decode as string 
        image_decoded = tf.image.decode_jpeg(image_string, channels=3)
        # image_decoded = tf.image.decode_gif(image_string)
        ## P.S tf.image.decode_image return shapeless tensor.
        # image_decoded = tf.image.decode_image(image_string)
        
        # normailization
        image_normalized = (tf.cast(image_decoded, tf.float32)/127.5) - 1
        # resize to given image size
        image_resized = tf.image.resize(image_normalized, (IMAGE_SIZE, IMAGE_SIZE))
        return image_resized, label
        """image_casted = tf.cast(image_decoded, tf.float32)
        image_resized = tf.image.resize(image_casted, (IMAGE_SIZE, IMAGE_SIZE))
        image_preproc = tf.keras.applications.inception_v3.preprocess_input(image_resized)
        
        return image_preproc, label"""
    
    except Exception as e:
        print(e)
        print(filename)

In [76]:
train_data = (train_data.map(_parse_fn)\
            # .shuffle(buffer_size=5000)\
            .batch(BATCH_SIZE)
             )

val_data = (val_data.map(_parse_fn)
           # .shuffle(buffer_size=5000)
           .batch(BATCH_SIZE)
           )

## Define the model

In [77]:
# Base model with MobileNetV2
IMG_SHAPE = (IMAGE_SIZE, IMAGE_SIZE, 3)

base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')
base_model.trainable = False

mobilenetV2 = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False, 
                                               weights='imagenet')
mobilenetV2.trainable = False

/home/ccuulinay/anaconda3/envs/tf2_0/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


In [78]:
# print(lv0_n_classes)

In [79]:
# Trainable classification head
maxpool_layer = tf.keras.layers.GlobalMaxPooling2D()
avgpool_layer = tf.keras.layers.GlobalAveragePooling2D()
dense_layer = tf.keras.layers.Dense(1024, activation='relu')
dropout_layer = tf.keras.layers.Dropout(0.5)
prediction_layer = tf.keras.layers.Dense(lv0_n_classes, activation='softmax')

learning_rate = 1e-4

model = tf.keras.Sequential([
    # base_model
    mobilenetV2
    , avgpool_layer
    # , maxpool_layer
    # , dense_layer
    # , dropout_layer
    , prediction_layer
])

# Now the optimizer is under tf.keras.
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=learning_rate)
    # optimizer=tf.keras.optimizers.SGD(lr=learning_rate, momentum=0.9)
    , loss='categorical_crossentropy'
    , metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_7 ( (None, 1280)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 5124      
Total params: 2,263,108
Trainable params: 5,124
Non-trainable params: 2,257,984
_________________________________________________________________


## Train model

In [80]:
num_epochs = 30
steps_per_epoch = round(len(train_filenames))//BATCH_SIZE
val_steps = 20

In [81]:
history = model.fit(train_data.repeat(),
                    epochs=num_epochs,
                    steps_per_epoch = steps_per_epoch,
                    validation_data=val_data.repeat(), 
                    validation_steps=val_steps)

Epoch 1/30
317/317 [==============================] - 60s 189ms/step - loss: 10.8390 - accuracy: 0.1607 - val_loss: 11.7201 - val_accuracy: 0.2109
Epoch 2/30
317/317 [==============================] - 58s 181ms/step - loss: 10.6787 - accuracy: 0.1718 - val_loss: 11.3149 - val_accuracy: 0.2109
Epoch 3/30
317/317 [==============================] - 57s 180ms/step - loss: 10.5950 - accuracy: 0.1770 - val_loss: 11.0566 - val_accuracy: 0.2016
Epoch 4/30
317/317 [==============================] - 57s 180ms/step - loss: 10.5347 - accuracy: 0.1798 - val_loss: 10.8963 - val_accuracy: 0.2078
Epoch 5/30
317/317 [==============================] - 57s 180ms/step - loss: 10.5077 - accuracy: 0.1834 - val_loss: 10.7719 - val_accuracy: 0.2094
Epoch 6/30
317/317 [==============================] - 57s 180ms/step - loss: 10.4801 - accuracy: 0.1867 - val_loss: 10.6811 - val_accuracy: 0.2094
Epoch 7/30
317/317 [==============================] - 57s 180ms/step - loss: 10.4583 - accuracy: 0.1894 - val_loss: 10

KeyboardInterrupt: 